<a href="https://colab.research.google.com/github/DAVID316CORDOVA/Air_Pollution_Forecasting_LSTM/blob/master/Tarea_1_Practica_distancia_minima_edicion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Aspectos generales

El autocorrector de errores ortograficos se usa todos los días en los computadores y celulares. En este taller, exploramos alguno de los principales aspectos de como están construidos con el Procesamiento de Lenguaje Natural y la distancia de edición mínima. No se revisan caracteristicas de despliegue o de ingeniería de sistemas para la construcción de una aplicación.

Los objetivos del taller son:
1. Implementar el conteo de palabras dado un corpus.
2. Obtener la probabilidad por palabra dado un corpus.
3. Manipular cadenas de texto.
4. Hacer filtos de cadenas de texto.
5. Implementar la Distancia de edición minima para comparar cadenas de texto y encotrar el paso optimo de edición.
6. Entender como funciona a la programación dinamica aplicada al PLN.

##Ejemplo de que es un autocorrector

El autocorrector de ortografía es como se muestra en la Figura 1

<img src = "https://drive.google.com/uc?id=1lXZbG4KjkrPyQ5JtzVaMKrQh86SY6xVf">

##Distancias de edición
En este taller se muestra la implementación de un autocorrect que esta a una (1) o dos (2) distancias de edición

Un distación de edición consiste en una de las siguientes opciones:


*   Borrar: oír => "ír, oí, or"
*   intercambio de dos letras adjacentes: comer => "ocmer, comre..."

*   Replazar: fuego => "juego, luego, ruego..."
*   Insertar, agregar una letra: ir => "ire, ira, ird, air..."





##Bayes Rules

El objetivo es hacer una autocorrección sugerida por la probabilidad dado un corpus:

$$P(c|w) = \frac{P(w|c)\times P(c)}{P(w)} \tag{Eqn-1}$$

La ecuación 1 describe la [Bayes Rule](https://en.wikipedia.org/wiki/Bayes%27_theorem)

#Preprocesamiento

In [1]:
#se importan las librerias requeridas
import re
from collections import Counter
import numpy as np
import pandas as pd

In [2]:
#cargue del corpus

def process_data(file_name):
    """
    Input:
        A file_name which is found in your current directory. You just have to read it in.
    Output:
        words: a list containing all the words in the corpus (text file you read) in lower case.
    """
    words = [] # return this variable correctly

    #Open the file, read its contents into a string variable
    with open(file_name, "r") as file:
        texto = file.read()

    # convert all letters to lower case
    texto = texto.lower()
    #Convert every word to lower case and return them in a list.
    words = re.findall("\w+", texto)

    return words

In [4]:
words = process_data("/content/elquijote.txt")
print("el numero de palabras es: {}".format(len(words)))
vocab = set(words)
print("el vocabulario (numero de palabras unicas) es de: {}".format(len(vocab)))
print("algunas de las palabras son: {}".format(words[500:550]))

el numero de palabras es: 384442
el vocabulario (numero de palabras unicas) es de: 23631
algunas de las palabras son: ['día', 'de', 'la', 'data', 'desta', 'nuestra', 'cédula', 'so', 'pena', 'que', 'la', 'persona', 'o', 'personas', 'que', 'sin', 'tener', 'vuestro', 'poder', 'lo', 'imprimiere', 'o', 'vendiere', 'o', 'hiciere', 'imprimir', 'o', 'vender', 'por', 'el', 'mesmo', 'caso', 'pierda', 'la', 'impresión', 'que', 'hiciere', 'con', 'los', 'moldes', 'y', 'aparejos', 'della', 'y', 'más', 'incurra', 'en', 'pena', 'de', 'cincuenta']


##Diccionario

Crear el disccionario, donde las llaves son las palabras y los valores es el numero de veces que la palabra aparece

<img src = "https://drive.google.com/uc?id=1bi25J511Uaw4Gwr_Z7L6FUII5zbmP6I_">

In [6]:
def get_count(word_l):
    '''
    Input:
        word_l: a set of words representing the corpus.
    Output:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    '''

    word_count_dict = {}  # Creamos un diccionario vacío para almacenar la frecuencia de cada palabra

    ### START CODE HERE ###
    for word in word_l:  # Iteramos por cada palabra en el conjunto de palabras
        if word in word_count_dict:
            word_count_dict[word] += 1  # Si la palabra ya está en el diccionario, incrementamos su contador
        else:
            word_count_dict[word] = 1   # Si es la primera vez que vemos la palabra, la agregamos con valor 1
    ### END CODE HERE ###

    return word_count_dict

In [ ]:
word_count_dict = get_count(words)
print("existen {} keys".format(len(word_count_dict)))
print("las veces que se menciona dulcinea: {}".format(word_count_dict.get("dulcinea", 0)))

existen 23631 keys
las veces que se menciona dulcinea: 282


##Calcular las probabilidades

Dado el diccionario, se deben calcular las probabilidades que una palabra sea seleccionada de forma aleatoria dado el corpus

$$P(w_i) = \frac{C(w_i)}{M} \tag{Eqn-2}$$

Donde $C(w_i)$ es el numero de veces $w_i$ aparece en el corpus.

$M$ es el numero total de palabras en el corpus

Por ejemplo, la probabilidad que la palabra nlp aparece en la oración "estoy en la clase de nlp" es

$$P(am) = \frac{C(w_i)}{M} = \frac {2}{12} \tag{Eqn-3}.$$

In [ ]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur.
    '''
    probs = {}  # return this variable correctly

    ### START CODE HERE ###

    # get the total count of words for all words in the dictionary



    ### END CODE HERE ###
    return probs

In [ ]:
probs = get_probs(word_count_dict)
print("numero de total de palabras: {}".format(len(probs)))
print("P(dulcinea) es: {:.4f}".format(probs["dulcinea"]))

numero de total de palabras: 23631
P(dulcinea) es: 0.0007


#Manipulación de cadenas de caracteres - Distancias de edición:

##List comprehensions
La manipulación de cadenas de caracteres y lista en python se sugiere realizar a traves de funciones denominadas [list comprehensions](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions)

Python List Comprehensions es un loop embebido dentro de una estructura de lista, colapsando muchas lineas de codigos en una sola.

La figura 2 muesta un loop tradicional y una list comprehension

<img src = "https://drive.google.com/uc?id=1sSmz8YIMFhp4jopHE1Nfeq2vrjmi7R8X"/>
**Figura 2**


##Borrar

**Instrucciones**
Crear la función delete_letter()

Por ejemplo, dada la palabra **oir**, retornar un conjuto de {"ir", "oi", "or"}

**Paso 1**
Dividir la palabra por caracteres de forma secuancial de izquierda a derecha. Por ejemplo:


```
[("", "oir"), ("o", "ir"), ("oi", "r"), ("oir","")]
```



Ver ejemplo como se muestra en la figura 3

<img src = "https://drive.google.com/uc?id=1hDW7M1hnBfkrV9Qnjap5Yl0F_rcX7v6H"/>
 **Figura 3**

Este es un paso común a todas las funciones delete_letter, switch_letter, replace_letter, insert_letter

**Paso 2**
Generar todas las palabras que resultan de borrar un caracter. Por ejemplo

```
["ir", "or", "oi"]
```
La figura 4 muestra una comparación para realizar el loop de formal generica o usando listh comprehensions

<img src = "https://drive.google.com/uc?id=1XzzMTmdXJEwm3m3E_CSPludJq7_CujGo"/> **Figura 4**



In [ ]:
def delete_letter(word, verbose=False):
    '''
    Input:
        word: the string/word for which you will generate all possible words
                in the vocabulary which have 1 missing character
    Output:
        delete_l: a list of all possible strings obtained by deleting 1 character from word
    '''

    delete_l = []
    split_l = []

    ### START CODE HERE ###
    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    delete_l = [L + R[1:] for L, R in split_l if R]
    ### END CODE HERE ###

    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return  delete_l

In [ ]:
delete_word_l = delete_letter(word="oir", verbose=True)

input word oir, 
split_l = [('', 'oir'), ('o', 'ir'), ('oi', 'r'), ('oir', '')], 
delete_l = ['ir', 'or', 'oi']


##Intercambio

**Instrucciones**
Crear la función switch_letter()

Por ejemplo, dada la palabras comer, retornar {"ocmer, comre..."}, pero NO se intercambian palabras que no sean adyacentes como: "eomcr"

**Paso 1** Implementer la función split. Es la misma que en la fución delete_letter()

**Paso 2** implementar una list comprehension que intercambie letras adyacentes

In [ ]:
def switch_letter(word, verbose=False):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent charater switched
    '''

    switch_l = []
    split_l = []

    ### START CODE HERE ###


    ### END CODE HERE ###

    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}")

    return switch_l

In [ ]:
switch_word_l = switch_letter(word="comer", verbose=True)

Input word = comer 
split_l = [('', 'comer'), ('c', 'omer'), ('co', 'mer'), ('com', 'er'), ('come', 'r'), ('comer', '')] 
switch_l = ['ocmer', 'cmoer', 'coemr', 'comre']


##Reemplazar

**Instrucciones**
Crear la función replace_letter()
Dada una palabra como fuego, retornar palabras como "juego, luego, ruego..."

**Paso 1** Implementer la función split. Es la misma que en la fución delete_letter()

**Paso 2** Implementar una List comprehension que reemplace las letras.

Puede ser de la forma siguiente:

```
[f(a, b, c) for a, b in splits if condition for c in string]
```

Se espera que la palabra original quede incluída. Por ejemplo al reeamplazar la primera letra de "fuego" con "f" retornará "fuego



**Paso 3** Remover la palabra original del conjunto de palabras

In [ ]:
def replace_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word.
    '''

    letters = 'abcdefghijklmnopqrstuvwxyz'

    replace_l = []
    split_l = []

    ### START CODE HERE ###




    ### END CODE HERE ###

    # turn the set back into a list and sort it, for easier viewing
    replace_l = sorted(list(replace_set))


    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")

    return replace_l

In [ ]:
replace_l = replace_letter(word='fuego', verbose=True)

##Insertar

**Instrucciones**
Crear la función insertar_letter()

**Paso 1** Implementer la función split. Es la misma que en la fución delete_letter()

**Paso 2** Implementar una List comprehensions que inserte las letras. Puede tener la siguiente forma:



```
[f(a, b, c) for a, b in splits if conditions for c in string]
```



In [ ]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    '''
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []

    ### START CODE HERE ###



    ### END CODE HERE ###

    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")

    return insert_l

In [ ]:
insert_l = insert_letter('ir', True)

Input word ir 
split_l = [('', 'ir'), ('i', 'r'), ('ir', '')] 
insert_l = ['air', 'bir', 'cir', 'dir', 'eir', 'fir', 'gir', 'hir', 'iir', 'jir', 'kir', 'lir', 'mir', 'nir', 'oir', 'pir', 'qir', 'rir', 'sir', 'tir', 'uir', 'vir', 'wir', 'xir', 'yir', 'zir', 'iar', 'ibr', 'icr', 'idr', 'ier', 'ifr', 'igr', 'ihr', 'iir', 'ijr', 'ikr', 'ilr', 'imr', 'inr', 'ior', 'ipr', 'iqr', 'irr', 'isr', 'itr', 'iur', 'ivr', 'iwr', 'ixr', 'iyr', 'izr', 'ira', 'irb', 'irc', 'ird', 'ire', 'irf', 'irg', 'irh', 'iri', 'irj', 'irk', 'irl', 'irm', 'irn', 'iro', 'irp', 'irq', 'irr', 'irs', 'irt', 'iru', 'irv', 'irw', 'irx', 'iry', 'irz']


#Combinación de ediciones

La función intercambio no es muy común por lo cual se deja la opción de imprimir o no

##edición de una letra

In [ ]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Please return a set. and not a list.
    """

    edit_one_set = set()

    ### START CODE HERE ###
    edit_one_set.update(delete_letter(word))

    if allow_switches:
        edit_one_set.update(switch_letter(word))

    edit_one_set.update(replace_letter(word))

    edit_one_set.update(insert_letter(word))
    ### END CODE HERE ###

    # return this as a set and not a list
    return edit_one_set

In [ ]:
tmp_word = "ir"
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))
print(tmp_edit_one_l)

['air', 'ar', 'bir', 'br', 'cir', 'cr', 'dir', 'dr', 'eir', 'er', 'fir', 'fr', 'gir', 'gr', 'hir', 'hr', 'i', 'ia', 'iar', 'ib', 'ibr', 'ic', 'icr', 'id', 'idr', 'ie', 'ier', 'if', 'ifr', 'ig', 'igr', 'ih', 'ihr', 'ii', 'iir', 'ij', 'ijr', 'ik', 'ikr', 'il', 'ilr', 'im', 'imr', 'in', 'inr', 'io', 'ior', 'ip', 'ipr', 'iq', 'iqr', 'ira', 'irb', 'irc', 'ird', 'ire', 'irf', 'irg', 'irh', 'iri', 'irj', 'irk', 'irl', 'irm', 'irn', 'iro', 'irp', 'irq', 'irr', 'irs', 'irt', 'iru', 'irv', 'irw', 'irx', 'iry', 'irz', 'is', 'isr', 'it', 'itr', 'iu', 'iur', 'iv', 'ivr', 'iw', 'iwr', 'ix', 'ixr', 'iy', 'iyr', 'iz', 'izr', 'jir', 'jr', 'kir', 'kr', 'lir', 'lr', 'mir', 'mr', 'nir', 'nr', 'oir', 'or', 'pir', 'pr', 'qir', 'qr', 'r', 'ri', 'rir', 'rr', 'sir', 'sr', 'tir', 'tr', 'uir', 'ur', 'vir', 'vr', 'wir', 'wr', 'xir', 'xr', 'yir', 'yr', 'zir', 'zr']


##edición de dos letras

In [ ]:
def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''

    edit_two_set = set()

    ### START CODE HERE ###
    one_set = edit_one_letter(word, allow_switches = allow_switches)
    for word in one_set:
        if word:
            edit_two = edit_one_letter(word, allow_switches = allow_switches)
            edit_two_set.update(edit_two)

    ### END CODE HERE ###

    # return this as a set instead of a list
    return set(edit_two_set)

In [ ]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(tmp_edit_two_l)

['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag', 'aah', 'aai', 'aaj', 'aak', 'aal', 'aam', 'aan', 'aao', 'aap', 'aaq', 'aar', 'aas', 'aat', 'aau', 'aav', 'aaw', 'aax', 'aay', 'aaz', 'ab', 'aba', 'abb', 'abc', 'abd', 'abe', 'abf', 'abg', 'abh', 'abi', 'abj', 'abk', 'abl', 'abm', 'abn', 'abo', 'abp', 'abq', 'abr', 'abs', 'abt', 'abu', 'abv', 'abw', 'abx', 'aby', 'abz', 'ac', 'aca', 'acb', 'acc', 'acd', 'ace', 'acf', 'acg', 'ach', 'aci', 'acj', 'ack', 'acl', 'acm', 'acn', 'aco', 'acp', 'acq', 'acr', 'acs', 'act', 'acu', 'acv', 'acw', 'acx', 'acy', 'acz', 'ad', 'ada', 'adb', 'adc', 'add', 'ade', 'adf', 'adg', 'adh', 'adi', 'adj', 'adk', 'adl', 'adm', 'adn', 'ado', 'adp', 'adq', 'adr', 'ads', 'adt', 'adu', 'adv', 'adw', 'adx', 'ady', 'adz', 'ae', 'aea', 'aeb', 'aec', 'aed', 'aee', 'aef', 'aeg', 'aeh', 'aei', 'aej', 'aek', 'ael', 'aem', 'aen', 'aeo', 'aep', 'aeq', 'aer', 'aes', 'aet', 'aeu', 'aev', 'aew', 'aex', 'aey', 'aez', 'af', 'afa', 'afb', 'afc', 'afd', 'afe', 'aff', 'a

#Sugerencias de corrección

In [ ]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    '''
    Input:
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output:
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''

    suggestions = []
    n_best = []

    ### START CODE HERE ###
    #Step 1: create suggestions as described above


    #Step 2: determine probability of suggestions

    #Step 3: Get all your best words and return the most probable top n_suggested words as n_best


    ### END CODE HERE ###

    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best

In [ ]:
my_word = 'dolcinea'
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) # keep verbose=True
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

print("data type of corrections {}".format(type(tmp_corrections)))

entered word =  dolcinea 
suggestions =  ['dulcinea']
word 0: dulcinea, probability 0.000734
data type of corrections <class 'list'>
